In [198]:

import sys
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import SelectKBest, f_regression, chi2, mutual_info_regression, SelectPercentile, VarianceThreshold
from sklearn.ensemble import VotingRegressor
from sklearn.preprocessing import MinMaxScaler
from joblib import Parallel, delayed
import numpy as np
from scipy.stats import pearsonr
import multiprocessing
from scipy.cluster import hierarchy
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from collections import Counter


sys.path.append('../../../')

from src.emotion.prediction.aggregates.train import HyperparaSearch
from src.emotion.prediction.aggregates.models import MODELS
from src.emotion.utils.constants import DATA_DIR

%matplotlib inline

In [199]:
features = pd.read_csv('/home/moritz/Workspace/masterthesis/data/features_dataset.csv')

In [200]:
targets = pd.read_csv('/home/moritz/Workspace/masterthesis/data/perma_scores_dataset.csv')

In [201]:
df = pd.merge(features, targets, on=["E-Mail-Adresse", "Day"])

df.shape

(104, 9351)

In [202]:
# Handle Missing Values

df.dropna(axis=1, how='any', inplace=True)
#df = dataset.loc[:, (df != 0).any(axis=0)]

print(df.shape)


(104, 9348)


In [203]:
# Detect outliers

# Check if all PERMA values are the same in each row
same_PERMA = (df['P'] == df['E']) & (df['E'] == df['R']) & (df['R'] == df['M']) & (df['M'] == df['A'])
# Remove the rows where all PERMA values are the same
df = df[~same_PERMA]
print(df.shape)

# find columns where all values are the same
cols_to_drop = [col for col in df.columns if df[col].nunique() == 1]
# drop the columns
df = df.drop(cols_to_drop, axis=1)
print(df.shape)

# drop columns where all values are only 0 or 1
df = df.loc[:, ~(df.isin([0, 1]).all() & ~df.isin([0, 1]).any())]
print(df.shape)

(104, 9348)
(104, 8759)
(104, 8759)


In [ ]:
# Load X and Y
# Store the PERMA values in Y
Y = df[['P', 'E', 'R', 'M', 'A']]

# Store the other columns in X
X = df.drop(columns=['ClassID', 'E-Mail-Adresse', 'Day', 'First Name', 'Last Name/Surname', 'P', 'E', 'R', 'M', 'A'])

In [ ]:
# Scale Features

# Create a MinMaxScaler object
scaler = MinMaxScaler()

# Fit the scaler to the dataframe and transform the dataframe
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)


In [ ]:
X.shape

In [ ]:
corr_matrix = X.corr()

In [ ]:
def plot_correlation_matrix(matrix):
    # center the matrix
    matrix = matrix - np.mean(matrix, axis=0)

    # transpose the matrix
    matrix_t = matrix.T

    # compute the correlation matrix using np.corrcoef
    corr_matrix = np.corrcoef(matrix_t)

    # create a heatmap of the correlation matrix using seaborn
    sns.set(font_scale=0.7)
    sns.heatmap(corr_matrix, cmap="YlGnBu")

In [ ]:
#plot_correlation_matrix(X)

In [ ]:
# Feature selection: runs in ~ 5min 
# Step 1: Identify feature clusters
# Create a dendrogram using hierarchical clustering
linkage = hierarchy.linkage(corr_matrix, method='complete')
plt.figure(figsize=(10, 5))
plt.title('Dendrogram')
plt.xlabel('Data points')
plt.ylabel('Distance')
hierarchy.dendrogram(
    linkage,
    leaf_rotation=0.,  # Rotate x-axis labels
    leaf_font_size=12.,  # Font size for x-axis labels
)
plt.show()

In [ ]:
# Get clusters from the dendrogram
max_d = 15 # Maximum distance between clusters
clusters = hierarchy.fcluster(linkage, max_d, criterion='distance')

clusters

In [ ]:
# Group columns by cluster
df = pd.DataFrame(corr_matrix)
df.columns = ['col_' + str(i) for i in range(df.shape[1])]
df['cluster'] = clusters
grouped = df.groupby('cluster')

# Get the size of each group
group_sizes = grouped.size()

# Plot the group sizes
plt.figure(figsize=(10,5))
plt.bar(x=group_sizes.index, height=group_sizes.values, color='blue')
plt.title('Group Sizes')
plt.xlabel('Group')
plt.ylabel('Size')
plt.show()

In [ ]:
print(Y.shape)
print(X.shape)
print(len(clusters))

In [ ]:
# Step 3: Calculate the correlation matrix between the columns of X and the columns of Y
corr_matrix = np.abs(np.corrcoef(X.T, Y.T)[:X.shape[1], X.shape[1]:])

# compute the row-wise averages of the matrix
avg_matrix = np.mean(corr_matrix, axis=1, keepdims=True)

# concatenate the average matrix with the group array
concat_matrix = np.concatenate([avg_matrix, clusters.reshape(len(clusters), 1)], axis=1)

# sort the concatenated matrix by group
sorted_matrix = concat_matrix[concat_matrix[:, -1].argsort()]

# find the maximum value in each group and its index
max_values = []
# iterate over the unique groups in the second column of the sorted matrix
for group in np.unique(sorted_matrix[:, 1]):
    # find the indices of rows that belong to the current group
    indices = np.where(sorted_matrix[:, 1] == group)[0]
    # get the maximum value in the first column for the current group
    max_value = np.max(sorted_matrix[indices, 0])
    # append the maximum value to the list
    max_values.append(max_value)
    
    
# find the indices of all the maximum values in the avg_matrix
max_indices = []
for max_value in max_values:
    indices = np.where(avg_matrix == max_value)[0]
    max_indices.extend(indices)

X_filtered = X.iloc[:, max_indices]
print(X_filtered.shape)



In [ ]:
plot_correlation_matrix(X_filtered)

In [ ]:
# # Ensemble feature selection (using voting) across multiple filter methods
# # TODO: Maybe create seperate feature sets for each PERMA dimension seperatly?
# def get_selected_features(Y, X_filtered, filter_methods):
#     selected_features = []
    
#     for i in range(Y.shape[1]):
#         y_i = Y.iloc[:, i]
#         selected_i = []
        
#         for method_name, method in filter_methods.items():
#             pipeline = Pipeline([(method_name, method), ('regressor', LinearRegression())])
#             pipeline.fit(X_filtered, y_i)
#             selected_i.append(pipeline.named_steps[method_name].get_support(indices=True))
        
#         selected_i = np.concatenate(selected_i)
#         selected_i = np.unique(selected_i)
#         selected_features.append(selected_i)
        
#     return np.concatenate(selected_features)

# filter_methods = {
#     'f_regression': SelectKBest(f_regression, k=1),
#     'mutual_info_regression': SelectKBest(mutual_info_regression, k=1),
#     'variance_threshold': VarianceThreshold(threshold=0.1),
# }

# all_selected_features = get_selected_features(Y, X_filtered, filter_methods)

# print("Fused selected features:", all_selected_features)




In [ ]:
def get_selected_voting_features_multi(Y, X_filtered, filter_methods, k):
    feature_counts = np.zeros(X_filtered.shape[1])
    
    #print(Y.shape[1])
    print(len(Y.T))
    
    for i in range(len(Y.T)):
        y_i = Y.iloc[:, i]
        
        for method_name, method in filter_methods.items():
            pipeline = Pipeline([(method_name, method), ('regressor', LinearRegression())])
            pipeline.fit(X_filtered, y_i)
            selected_i = pipeline.named_steps[method_name].get_support(indices=True)
            
            # Increment the count for each selected feature
            for index in selected_i:
                feature_counts[index] += 1
                
    # Get the indices of the top k features with the most counts
    top_k_features = np.argsort(feature_counts)[-k:]
    
    return top_k_features

In [ ]:
filter_methods = {
    'f_regression': SelectKBest(f_regression, k=10),
    'mutual_info_regression': SelectKBest(mutual_info_regression, k=10),
    'variance_threshold': VarianceThreshold(threshold=0.1),
}

all_selected_features = get_selected_voting_features_multi(Y, X_filtered, filter_methods, k=15)

print("Fused selected features:", all_selected_features)


In [ ]:
X_final = X_filtered.iloc[:, list(set(all_selected_features))]
plot_correlation_matrix(X_final)
column_names = X_final.columns.tolist()
print(column_names)

In [ ]:
models_to_drop = ["MLPRegressor", "KNeighborsRegressor", "DecisionTreeRegressor", "GradientBoostingRegressor", "SVR"]

for name in models_to_drop:
    for i in range(len(MODELS)):
        if MODELS[i]["name"] == name:
            del MODELS[i]
            break
        
for model in MODELS:
    print(model["name"])

In [ ]:
# Runs in ~5 min for n_fols=5
search = HyperparaSearch(models=MODELS, metrics=["mean_absolute_error"], n_folds=5, n_jobs=-1)

results = search.run(X_final, Y, save=False)

In [ ]:
best_feats_dict = {model[0]['name']: model[0]['best_feats'] for model in results}
#print(best_feats_dict)

In [ ]:
for model in best_feats_dict.keys():
    print(f"Algorithm: {model}")
    # Get the feature importance values for the algorithm
    feat_imp_vals = best_feats_dict[model]
    # Map the feature importance values with the feature list using a dictionary comprehension
    feat_imp_map = {column_names[i]: feat_imp_vals[i] for i in range(len(column_names))}
    # Rank the features by their importance value in descending order
    ranked_feats = sorted(feat_imp_map.items(), key=lambda x: x[1], reverse=True)
    # Print the ranked features
    #print(ranked_feats)

In [ ]:
eval_metric = "mean_absolute_error"

# Plot the results
mae_scores = [
    rd["score"]
    for result_list in results
    for rd in result_list
    if rd["metric"] == eval_metric
]
model_names = [
    rd["name"]
    for result_list in results
    for rd in result_list
    if rd["metric"] == eval_metric
]
plt.bar(model_names, mae_scores)
plt.title("Mean Absolute Error Scores")
plt.xlabel("Model")
plt.ylabel("Score")
plt.xticks(rotation=45) 
plt.show()

In [ ]:
# Print the model with the lowest score
best_model = min(
    [
        min(
            sublist,
            key=lambda x: x["score"]
            if x["metric"] == eval_metric
            else float("inf"),
        )
        for sublist in results
    ],
    key=lambda x: x["score"],
)
print(f"Best model: {best_model['name']}")
print(f"Best params: {best_model['params']}")
print(f"Best Score: {best_model['score']}")

In [ ]:
def get_selected_voting_features_uni(Y, X_filtered, filter_methods, k):
    feature_counts = np.zeros(X_filtered.shape[1])
        
    for method_name, method in filter_methods.items():
        pipeline = Pipeline([(method_name, method), ('regressor', LinearRegression())])
        pipeline.fit(X_filtered, Y)
        selected_i = pipeline.named_steps[method_name].get_support(indices=True)
        
        # Increment the count for each selected feature
        for index in selected_i:
            feature_counts[index] += 1
                
    # Get the indices of the top k features with the most counts
    top_k_features = np.argsort(feature_counts)[-k:]
    
    return top_k_features

In [ ]:
# Create a dictionary for each PERMA pillar
perma_dict = {
    "P": None,
    "E": None,
    "R": None,
    "M": None,
    "A": None,
}

filter_methods = {
    'f_regression': SelectKBest(f_regression, k=10),
    'mutual_info_regression': SelectKBest(mutual_info_regression, k=10),
    'variance_threshold': VarianceThreshold(threshold=0.1),
}

# Iterate over each PERMA dimension in Y
for i, perma_dim in enumerate(Y.columns):
    # Get the selected features for the current PERMA dimension
    selected_features = get_selected_voting_features_uni(Y.iloc[:, i], X_filtered, filter_methods, k=10)
    # Save the selected features in the corresponding dictionary for the current PERMA pillar
    perma_dict[perma_dim] = selected_features

# Print the selected features for each PERMA dimension in each PERMA pillar
for pillar in perma_dict:
    print(pillar, ":", perma_dict[pillar])


In [ ]:
feature_sets = {}

for perma_dim, selected_features in perma_dict.items():
    # Select the corresponding columns of X_filtered
    X_final = X_filtered.iloc[:, list(set(selected_features))]
    # Add the selected features for the current PERMA dimension to the feature sets dictionary
    feature_sets[perma_dim] = X_final
    # Print the names of the selected features
    column_names = X_final.columns.tolist()
    print(perma_dim, ":", column_names)


In [ ]:
MODELS = [
    {
        "name": "RandomForestRegressor",
        "model": RandomForestRegressor(),
        "params": {"n_estimators": [50, 100, 200, 400], "max_depth": range(2, 6)},
    },
    {
        "name": "ExtraTreesRegressor",
        "model": ExtraTreesRegressor(),
        "params": {"n_estimators": [50, 100, 200, 400], "max_depth": range(2, 6)},
    },
    # {
    #     "name": "GradientBoostingRegressor",
    #     "model": GradientBoostingRegressor(),
    #     "params": {
    #         "n_estimators": [50, 100, 200, 400],
    #         "max_depth": range(2, 6),
    #         "learning_rate": [0.001, 0.01, 0.1],
    #     },
    # },
    # {
    #     "name": "AdaBoostRegressor",
    #     "model": AdaBoostRegressor(),
    #     "params": {
    #         "n_estimators": [50, 100, 200, 400],
    #         "learning_rate": [0.001, 0.01, 0.1],
    #     },
    # },
    {
        "name": "LinearRegression",
        "model": LinearRegression(),
        "params": {},
    },
    {
        "name": "Ridge",
        "model": Ridge(),
        "params": {"alpha": [0.001, 0.01, 0.1, 1.0]},
    },
    {
        "name": "Lasso",
        "model": Lasso(),
        "params": {"alpha": [0.001, 0.01, 0.1, 1.0]},
    },
    {
        "name": "ElasticNet",
        "model": ElasticNet(),
        "params": {"alpha": [0.001, 0.01, 0.1, 1.0], "l1_ratio": [0.1, 0.5, 0.9]},
    },
    {
        "name": "BayesianRidge",
        "model": BayesianRidge(),
        "params": {
            "alpha_1": [0.001, 0.01, 0.1, 1.0],
            "alpha_2": [0.001, 0.01, 0.1, 1.0],
        },
    },
    # {
    #     "name": "CatBoostRegressor",
    #     "model": CatBoostRegressor(verbose=False),
    #     "params": {
    #         "iterations": [50, 100, 200, 400],
    #         "depth": range(2, 6),
    #         "learning_rate": [0.001, 0.01, 0.1, 1.0],
    #     },
    # },
    {
        "name": "XGBRegressor",
        "model": XGBRegressor(),
        "params": {
            "n_estimators": [50, 100, 200, 400],
            "max_depth": range(2, 6),
            "learning_rate": [0.001, 0.01, 0.1],
        },
    },
]


In [ ]:
for perma_dim, X_final in feature_sets.items():
    print(f"PERMA dimension: {perma_dim}")
    # Run the hyperparameter search
    search = HyperparaSearch(models=MODELS, metrics=["mean_absolute_error"], n_folds=5, n_jobs=-1, mode="uni", verbose=False)
    results = search.run(X_final, Y[perma_dim], save=False)
    # Print the model with the lowest score
    best_model = min(
        [
            min(
                sublist,
                key=lambda x: x["score"]
                if x["metric"] == eval_metric
                else float("inf"),
            )
            for sublist in results
        ],
        key=lambda x: x["score"],
    )
    print(perma_dim)
    print(f"Best model: {best_model['name']}")
    print(f"Best params: {best_model['params']}")
    print(f"Best Score: {best_model['score']}")